<a href="https://colab.research.google.com/github/naemililia/masterthesis/blob/main/pipeline_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is optimized for google colab

immer am anfang:
- alignment helpers hochladen, da dessen Funktionen genutzt werden



In [ ]:
# define which auxiliary information i want to create labels for now, and also which satellite source

UNLABELED_DATASET = 'Sen1floods11'
AUXILIARY_DATASOURCE = 'BUILT' # 'LC', 'BUILT'


In [ ]:
# installations
!pip install utm
!pip install rasterio

  Preparing metadata (setup.py) ... done
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6084 sha256=d798f71e6ddfca71a4e201f359bff5e9834cc711c52b13531916d4202724750b
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.3 MB/s eta 0:00:00


In [ ]:
import sys
import os
import shutil
from tqdm import tqdm
import utm
import rasterio
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import numpy as np
from pathlib import Path


In [ ]:
# import my own file with helper functions
sys.path.append('/content')
from alignment_helpers import *

In [ ]:
# mount drive to access aux data, and to save labels
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# download flood data
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_train_data.csv .
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_test_data.csv .
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_valid_data.csv .

Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_train_data.csv...
/ [1 files][ 13.3 KiB/ 13.3 KiB]                                                
Operation completed over 1 objects/13.3 KiB.                                     
Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_test_data.csv...
/ [1 files][  4.7 KiB/  4.7 KiB]                                                
Operation completed over 1 objects/4.7 KiB.                                      
Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_valid_data.csv...
/ [1 files][  4.7 KiB/  4.7 KiB]                                                
Operation completed over 1 objects/4.7 KiB.                                      


In [ ]:
# download all files into S1weak_all (we now want the weakly labeled satellite images, not the hand labeled ones)

!sudo mkdir files
#!sudo mkdir files/S1perm # decided against S1perm for now, as i want to use the flood data and ifnetune on flood data, + the perm data would mean that i need to download a lot more auxiliary info
!sudo mkdir files/S1weak_all

#!gsutil -m rsync -r gs://sen1floods11/v1.1/data/perm_water/S1Perm files/S1perm
!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak files/S1weak_all


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
Starting synchronization...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1163442_S1Weak.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1009032_S1Weak.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1264498_S1Weak.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1159714_S1Weak.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1041766_S1Weak.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/WeaklyLabeled/S1Weak/Bolivia_1037580_S1Weak.tif...
Copying gs://sen1floods11/v1.1

In [ ]:
# Create subfolders from the S1Weak folder of 500 images each to create batches for better handling and supervision (if connection is lost etc)

source_dir = '/content/files/S1weak_all'
files = os.listdir(source_dir)
batch_size = 500

for i in range(0, len(files), batch_size):
    # Create batch directory
    batch_dir = os.path.join('/content/files/S1weak', f'batch_{i//batch_size + 1}')
    os.makedirs(batch_dir, exist_ok=True)

    # Move files in batches
    for file in files[i:i+batch_size]:
        shutil.move(os.path.join(source_dir, file), batch_dir)

In [ ]:
def load_ghsl_shapefile():
  """
  This function loads the shapefile provided by the GHSL into a dataframe and adds the tile_ids (also provided by them) as a column
  """
  shapefile = gpd.read_file('/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/BUILT/shape/WGS84_tile_schema.shp')
  tile_ids = pd.read_csv('/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/BUILT/shape/WGS84_tile_schema.csv', header=None)
  shapefile['tile_id'] = tile_ids

  return shapefile

In [ ]:
def resize_the_label(array_label):
  """
  This function resized the labels to the size of the input images from the unlabeled data source.

  Depending on the auxiliary data, different orders of zoom are used.

  for BUILT and POP, both continuous, regression like variables, order = 1, making it a trilinear interpolation (https://labs.quansight.org/blog/2021/01/scipy-ndimage-interpolation) / bilinear interpolation (chatgpt)

  for LC, a variable with a discrete amount of classes, order = 0, making it a nearest neighbor resizing process
  """
  original_shape = array_label.shape

  if UNLABELED_DATASET == 'Sen1floods11':
    new_shape = (512, 512)
  elif UNLABELED_DATASET == 'EuroSAT':
    new_shape = (64, 64)

  zoom_factors = [n / o for n, o in zip(new_shape, original_shape)]

  if AUXILIARY_DATASOURCE == 'BUILT' or AUXILIARY_DATASOURCE == 'POP':
    order = 1
  elif AUXILIARY_DATASOURCE == 'LC':
    order = 0

  resized_label = zoom(array_label, zoom_factors, order = order)

  return resized_label


In [ ]:
def save_label_for_image(file, label_array, save_dir):
    """
    saves the label as a numpy array to the specified save_dir, using the name of the file, and appending '_label'
    """

    # Extract the base name without the extension
    base_name = file.stem
    label_filename = f"{base_name}_label.npy"
    label_path = os.path.join(save_dir, label_filename)
    np.save(label_path, label_array)

def save_resized_label_for_image(file, label_array, save_dir):
    """
    saves the resized label as a numpy array to the specified save_dir, using the name of the file, and appending '_label_resized'
    """
    # Extract the base name without the extension
    base_name = file.stem
    label_filename = f"{base_name}_label_resized.npy"
    label_path = os.path.join(save_dir, label_filename)
    np.save(label_path, label_array)

In [ ]:
def save_array_of_problematic_images(save_dir, list_of_files_in_multiple_tiles):
    """
    Some unlabeled satellite images are spread across multiple auxiliary tiles
    --> these ones are saved in a list, which is saved with this function in the same directory as the labels
    """
    with open(save_dir +'/list_of_files_in_multiple_tiles.txt', 'w') as f:
      for item in list_of_files_in_multiple_tiles:
          f.write("%s\n" % item)

In [ ]:
# PIPELINE

#test_path = Path('/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/flood')
base_path = Path('/content/files/S1weak/')
# initialize empty list to save the images which are spread across multiple aux info tiles
list_of_files_in_multiple_tiles = []
# initialize filecounter, used to print progress
file_counter = 0

for batch_folder in base_path.iterdir():
    if batch_folder.is_dir() and batch_folder.stem != '.ipynb_checkpoints':
      print(batch_folder.stem)
      for file in batch_folder.iterdir():
          if file.is_file() and file.suffix == '.tif':# and file.stem not in unique_filenames_list:
            # do some tracking
            print(file.stem)
            file_counter += 1
            if file_counter % 10 == 0:
                print(f'file {file_counter}')

            # read the raw satelite image and save its transform matrix and the array
            with rasterio.open(file) as src:
              array_sat = src.read(1)
              transform_sat = src.transform

              if UNLABELED_DATASET == 'EuroSAT':
                # if we have a eurosat image, then also save the coordinate system as a string, in order to extract the UTM zone
                crs_str = str(src.crs)

            # step 1: get the corner coordinates of the satellite image
            corner_coordinates = get_coordinates_of_corners(transform_sat, array_sat)
            if UNLABELED_DATASET == 'EuroSAT':
              # extract UTM zone and hemisphere first
              zone_nr, hemisphere = find_UTM_zone_eurosat(crs_str)
              corner_coordinates = get_coordinates_of_corners_from_UTM(zone_nr, corner_coordinates)

            # step 2: identify the correct tile of auxiliary info to download.
            # If the event is at the border of multiple tiles, then add the file name to a list, and skip the file for now
            if AUXILIARY_DATASOURCE == 'BUILT' or AUXILIARY_DATASOURCE == 'POP':
              shapefile = load_ghsl_shapefile()
              tile_id = get_BUILT_polygon_encircling_unlabeled_staelite_image(shapefile, corner_coordinates)
              if tile_id:
                file_name = 'GHS_' + AUXILIARY_DATASOURCE + '_E2020_GLOBE_R2023A_4326_3ss_V1_0_' + tile_id # wenn man es mit BUILT laufen lässt, dann noch _S_ vor E2020

              else:
                list_of_files_in_multiple_tiles.append(file)
                continue


            elif AUXILIARY_DATASOURCE == 'LC':
              tile_id = find_correct_land_cover_tile(corner_coordinates)
              if tile_id:
                file_name = tile_id + '_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326'
              else:
                list_of_files_in_multiple_tiles.append(file.stem)
                continue

            # step 3: load the correct tile of the auxiliary data source
            folder_name = '/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/'+ AUXILIARY_DATASOURCE + '/'
            with rasterio.open(folder_name + file_name + '.tif') as src:
              array_aux = src.read(1)
              transform_aux = src.transform

            # step 4: get the pixel coordinates for the geo-coordinates
            corner_pixels_aux = get_corner_pixels(corner_coordinates, transform_aux)

            # step 5: slice away the label
            label = get_label_from_pixel(corner_pixels_aux, array_aux)

            # save the 'raw' label to google drive
            save_dir = '/content/drive/My Drive/Uni/TUB/Semester/5 - Masterarbeit/data/'+AUXILIARY_DATASOURCE+ '/' + UNLABELED_DATASET
            save_label_for_image(file,label, save_dir)

            # step 6: resize the label
            resized_label = resize_the_label(label)

            # save the resized label to google drive
            save_resized_label_for_image(file,resized_label, save_dir)

      # save the problematic images to the same directory
      save_array_of_problematic_images(save_dir, list_of_files_in_multiple_tiles)






Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Somalia_151588_S1Weak
R9_C23 is the only tile to download
Colombia_9911520_S1Weak
R9_C12 is the only tile to download
Paraguay_2834197_S1Weak
R12_C13 is the only tile to download
Spain_1937976_S1Weak
R6_C18 is the only tile to download
Mekong_5207699_S1Weak
file 2010
R8_C29 is the only tile to download
Somalia_6438874_S1Weak
R9_C23 is the only tile to download
Ghana_2092661_S1Weak
R8_C18 is the only tile to download
Pakistan_697548_S1Weak
R6_C26 is the only tile to download
Sri-Lanka_3944881_S1Weak
R9_C27 is the only tile to download
Colombia_7150418_S1Weak
R9_C12 is the only tile to download
Pakistan_635539_S1Weak
R6_C26 is the only tile to download
Paraguay_7746346_S1Weak
R12_C13 is the only tile to download
Paraguay_3253224_S1Weak
R12_C13 is the only tile to download
India_3154844_S1Weak
R7_C28 is the only tile to download
Paraguay_6979890_S1Weak
file 2020
R12_C13 is the only tile to download
India_9468827_S1Weak
R7_

In [ ]:
# PIPELINE

#test_path = Path('/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/flood')
base_path = Path('/content/files/S1/')
# initialize empty list to save the images which are spread across multiple aux info tiles
list_of_files_in_multiple_tiles = []
# initialize filecounter, used to print progress
file_counter = 0


for file in batch_folder.iterdir():
    if file.is_file() and file.suffix == '.tif':# and file.stem not in unique_filenames_list:
      # do some tracking
      print(file.stem)
      file_counter += 1
      if file_counter % 10 == 0:
          print(f'file {file_counter}')

      # read the raw satelite image and save its transform matrix and the array
      with rasterio.open(file) as src:
        array_sat = src.read(1)
        transform_sat = src.transform

        if UNLABELED_DATASET == 'EuroSAT':
          # if we have a eurosat image, then also save the coordinate system as a string, in order to extract the UTM zone
          crs_str = str(src.crs)

      # step 1: get the corner coordinates of the satellite image
      corner_coordinates = get_coordinates_of_corners(transform_sat, array_sat)
      if UNLABELED_DATASET == 'EuroSAT':
        # extract UTM zone and hemisphere first
        zone_nr, hemisphere = find_UTM_zone_eurosat(crs_str)
        corner_coordinates = get_coordinates_of_corners_from_UTM(zone_nr, corner_coordinates)

      # step 2: identify the correct tile of auxiliary info to download.
      # If the event is at the border of multiple tiles, then add the file name to a list, and skip the file for now
      if AUXILIARY_DATASOURCE == 'BUILT' or AUXILIARY_DATASOURCE == 'POP':
        shapefile = load_ghsl_shapefile()
        tile_id = get_BUILT_polygon_encircling_unlabeled_staelite_image(shapefile, corner_coordinates)
        if tile_id:
          file_name = 'GHS_' + AUXILIARY_DATASOURCE + '_E2020_GLOBE_R2023A_4326_3ss_V1_0_' + tile_id # wenn man es mit BUILT laufen lässt, dann noch _S_ vor E2020

        else:
          list_of_files_in_multiple_tiles.append(file)
          continue


      elif AUXILIARY_DATASOURCE == 'LC':
        tile_id = find_correct_land_cover_tile(corner_coordinates)
        if tile_id:
          file_name = tile_id + '_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326'
        else:
          list_of_files_in_multiple_tiles.append(file.stem)
          continue

      # step 3: load the correct tile of the auxiliary data source
      folder_name = '/content/drive/MyDrive/Uni/TUB/Semester/5 - Masterarbeit/data/'+ AUXILIARY_DATASOURCE + '/'
      with rasterio.open(folder_name + file_name + '.tif') as src:
        array_aux = src.read(1)
        transform_aux = src.transform

      # step 4: get the pixel coordinates for the geo-coordinates
      corner_pixels_aux = get_corner_pixels(corner_coordinates, transform_aux)

      # step 5: slice away the label
      label = get_label_from_pixel(corner_pixels_aux, array_aux)

      # save the 'raw' label to google drive
      save_dir = '/content/drive/My Drive/Uni/TUB/Semester/5 - Masterarbeit/data/'+AUXILIARY_DATASOURCE+ '/' + UNLABELED_DATASET
      save_label_for_image(file,label, save_dir)

      # step 6: resize the label
      resized_label = resize_the_label(label)

      # save the resized label to google drive
      save_resized_label_for_image(file,resized_label, save_dir)

# save the problematic images to the same directory
save_array_of_problematic_images(save_dir, list_of_files_in_multiple_tiles)






Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Somalia_151588_S1Weak
R9_C23 is the only tile to download
Colombia_9911520_S1Weak
R9_C12 is the only tile to download
Paraguay_2834197_S1Weak
R12_C13 is the only tile to download
Spain_1937976_S1Weak
R6_C18 is the only tile to download
Mekong_5207699_S1Weak
file 2010
R8_C29 is the only tile to download
Somalia_6438874_S1Weak
R9_C23 is the only tile to download
Ghana_2092661_S1Weak
R8_C18 is the only tile to download
Pakistan_697548_S1Weak
R6_C26 is the only tile to download
Sri-Lanka_3944881_S1Weak
R9_C27 is the only tile to download
Colombia_7150418_S1Weak
R9_C12 is the only tile to download
Pakistan_635539_S1Weak
R6_C26 is the only tile to download
Paraguay_7746346_S1Weak
R12_C13 is the only tile to download
Paraguay_3253224_S1Weak
R12_C13 is the only tile to download
India_3154844_S1Weak
R7_C28 is the only tile to download
Paraguay_6979890_S1Weak
file 2020
R12_C13 is the only tile to download
India_9468827_S1Weak
R7_

In [ ]:
save_array_of_problematic_images(save_dir, list_of_files_in_multiple_tiles)
